# Meter Inventory Transformation for Import

### This notebook is a self-contained set of code and instructions that will enable you to convert individual files or batches of files from the vendor-provided format to the Milsoft-import format with minimal effort

## 0. Import packages
A few Python packages will be needed. Click the play button to import them for your session.

In [7]:
import os
import pandas as pd

## 1. Set the target folder or file
If you only want to convert one file, copy its filepath into the 'target_file' variable. If you want to convert all the .xlsx files in a folder, copy the folder path to the 'target_folder' variable. *NOTE* You MUST None-ify the variable you do NOT wish to use. Simply replace the undesired variable value with 'None' 

'output_folder' will be used as the destination for your converted file(s)

In [18]:
target_folder = "test_in"
target_file = None

output_folder = "test_out"

SIZE_TO_CODE = {
    '3"': "300",
    '5/8"': "058",
    '1"': "100",
    '1-1/2"': "150",
    '2"': "200",
    '4"': "400",
    '6"': "600"
}

## 2. Define Functions
You shouldn't mess with these unless you're familiar with Python and Pandas. Reach out to Connor McKinnis if changes are needed or if you suspect a function in here is misbehaving. 

In [16]:
def map_meter_size(size_str):
    if pd.isna(size_str):
        return ""
    for key, val in SIZE_TO_CODE.items():
        if key in size_str:
            return val
    return ""

def transform_excel_to_csv(input_path: str, output_path: str):
    # Load Excel data
    df = pd.read_excel(input_path)

    # Create transformed DataFrame
    output = pd.DataFrame()
    output["MISER"] = df["Meter SN"].fillna(df["Register SN"].astype(str))
    output["MISERV"] = 3
    output["MITSTDT"] = df["Meter Test Date"].fillna(df["Ship Date"])
    output["MIMTYPE"] = "Sensus"
    output["MIPDTE"] = "01/01/0001"
    output["MIMULT"] = 1
    output["MIKIND"] = "W"
    output["MIDIAL"] = 9
    output["MICDTE"] = "01/01/0001"
    output["MIMTRS"] = "IPK"
    output["MIHANDHELD"] = "N"
    output["MIOPEN2"] = df["Meter Size"].apply(map_meter_size)

    # Fill in missing required columns with blanks
    TARGET_COLUMNS = [
        'MISER', 'MISERV', 'MITSTDT', 'MITSTR', 'MIASFKWH', 'MIASLKWH', 'MIASFFL', 'MIASFLL',
        'MIASLLL', 'MIASLFL', 'MIWHY', 'MIMOD#', 'MIMTYPE', 'MIVOU', 'MIPDTE', 'MIPCST',
        'MIMULT', 'MIMNO', 'MIMFG', 'MITYPE', 'MIKIND', 'MIPHSE', 'MIVOLT', 'MIAMPS',
        'MIWIRE', 'MIRACTION', 'MIDIAL', 'MIFORM', 'MIRR', 'MICLAS', 'MIPF', 'MIOSER',
        'MIOREAD', 'MIOKWRD', 'MICDTE', 'MIWHSE', 'MIMCT', 'MIMTRS', 'MIOPEN2', 'MIOPEN3',
        'MIHANDHELD'
    ]

    for col in TARGET_COLUMNS:
        if col not in output.columns:
            output[col] = ""

    # Reorder the columns to match the required format
    output = output[TARGET_COLUMNS]

    # Save to CSV
    output.to_csv(output_path, index=False, encoding="utf-8")
    print(f"✅ Output saved to: {output_path}")
    
    
def process_file(file_path: str, output_folder: str):
    file = file_path if file_path.endswith('.xlsx') else None
    if file: 
        output_base = os.path.splitext(os.path.basename(file))[0]
        output_path = f"{output_folder}/{output_base}_transformed.csv"
        transform_excel_to_csv(file_path, output_path)
    else: 
        print("❌ No Excel file found. It must be a '.xlsx' file.")
        
def batch_process_folder(folder_path: str, output_folder:str):
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    if not files: 
        print("❌ No Excel files found in the specified folder.")
        return
    
    for file in files: 
        input_path = os.path.join(folder_path, file)
        output_base = os.path.splitext(os.path.basename(file))[0]
        output_path = f"{output_folder}/{output_base}_transformed.csv"
        transform_excel_to_csv(input_path, output_path)

## 3. Run
Use this to run your script.

In [19]:
if target_folder:
    batch_process_folder(target_folder, output_folder)
elif target_file:
    process_file(target_file, output_folder)
else: 
    print("Failed: You must populate a target_file OR target_folder in Step 1 AND replace the unused variable with 'None'")

✅ Output saved to: test_out/2025-00000399_transformed.csv
✅ Output saved to: test_out/2025-00000594_transformed.csv
✅ Output saved to: test_out/20251095_transformed.csv
✅ Output saved to: test_out/20251106_transformed.csv
✅ Output saved to: test_out/20251189_transformed.csv
✅ Output saved to: test_out/20251365_transformed.csv


C:\Users\LocalAdmin\AppData\Local\Temp\ipykernel_19192\2127259370.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output["MITSTDT"] = df["Meter Test Date"].fillna(df["Ship Date"])
